<img src="https://i.imgur.com/FoKB5Z5.png" align="left" width="300" height="250" title="source: imgur.com" /></a>

## Program Code: J620-002-4:2020 

## Program Name: FRONT-END SOFTWARE DEVELOPMENT

## Title :  Case Study - IMDB Web Scraping

#### Name: 

#### IC Number:

#### Date :

#### Introduction : 



#### Conclusion :






**Reference : https://medium.com/better-programming/the-only-step-by-step-guide-youll-need-to-build-a-web-scraper-with-python-e79066bd895a**

In [2]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import csv
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

## 1. Import Data by using webscrapping
Open the URL with headless webdriver and parse the page source into html with beautifulsoup

In [3]:
driver = webdriver.Chrome('C:\\Users\\Asus\\Documents\\ChromeDriver\\chromedriver\\chromedriver')

url = 'https://www.imdb.com/search/title/?groups=top_1000&ref_=adv_prvt'

driver.get(url)

Append data found into list according to the category

In [4]:
keepRunning = True
data = []
#  ranking title year rating duration genre imdb score metascore description director stars votes
while keepRunning:
    soup = BeautifulSoup(driver.page_source, "html.parser")
#     try to get the table and realize there is no table and only divs so looped on the div
    for div in soup.find_all('div', attrs={'class': 'lister-item mode-advanced'}):
#         realize there is only div so have to do from section to section this is the title one got rank title and year release i hate this
        for headerDetails in div.find_all('h3', attrs={'class':'lister-item-header'}):
            for header in headerDetails:
                if header.text.rstrip() != '':
                    data.append(header.text.rstrip())
                    
#         get the dum pg duration and genre
        if div.find('span',attrs={'class':'certificate'}) != None:
            data.append(div.find('span',attrs={'class':'certificate'}).text)
        else:
            data.append('Not Rated')
        data.append(div.find('span',attrs={'class':'runtime'}).text)
        data.append(div.find('span',attrs={'class':'genre'}).text.replace('\n', '').rstrip())
#         did the same as above and got the imdb score and meta score kill me man
        for scoreDetails in div.find_all('div', attrs={'class':'ratings-bar'}):
            data.append(scoreDetails.find('strong').text.rstrip())
            if scoreDetails.find('div',attrs={'class':'inline-block ratings-metascore'}) != None:
                data.append(scoreDetails.find('div',attrs={'class':'inline-block ratings-metascore'}).find('span').text.rstrip())
            else:
                data.append(np.nan)
        tempArr=[]
        for cast in div.find_all('p',attrs={'class':""}):
            if cast.text!="Director" or cast.text!="Stars":
                tempArr.append(cast.text)
        tempString=''.join(tempArr)
        tempdata=tempString.split('|')
        data.append(tempdata[0].replace('\n', '').replace('Directors:', '').replace('Director:', '').replace('    ','').rstrip())
        data.append(tempdata[1].replace('Stars:', '').replace(' \n', '').rstrip())
        data.append(div.find_all('p',attrs={'class':'text-muted'})[1].text.replace('\n', '').replace('   ','').rstrip())
        data.append(div.find('span',attrs={'name':'nv'}).text.replace('\n', '').rstrip())
        if len(div.find('p',attrs={'class':'sort-num_votes-visible'}).find_all('span')) >= 4:
            if div.find('p',attrs={'class':'sort-num_votes-visible'}).find_all('span')[3].text == 'Gross:':
                data.append(div.find_all('span',attrs={'name':'nv'})[1].text.replace('\n', '').rstrip())
            else:
                data.append(np.nan)
        else:
            data.append(np.nan)
            
    try:
        button = driver.find_element_by_class_name('lister-page-next.next-page')
        button.click()
    except NoSuchElementException:
        keepRunning = False

Check if the data is webscrapped successfully

In [5]:
data

['1.',
 'Spider-Man: Across the Spider-Verse',
 '(2023)',
 'PG',
 '140 min',
 'Animation, Action, Adventure',
 '8.9',
 '86',
 'Joaquim Dos Santos, Kemp Powers, Justin K. Thompson',
 '   Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez',
 'Miles Morales catapults across the Multiverse, where he encounters a team of Spider-People charged with protecting its very existence. When the heroes clash on how to handle a new threat, Miles must redefine what it means to be a hero.',
 '169,962',
 nan,
 '2.',
 'Titanic',
 '(1997)',
 'PG-13',
 '194 min',
 'Drama, Romance',
 '7.9',
 '75',
 'James Cameron',
 '   Leonardo DiCaprio,Kate Winslet,Billy Zane,Kathy Bates',
 'A seventeen-year-old aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic.',
 '1,228,149',
 '$659.33M',
 '3.',
 'Avatar: The Way of Water',
 '(2022)',
 'PG-13',
 '192 min',
 'Action, Adventure, Fantasy',
 '7.6',
 '67',
 'James Cameron',
 '   Sam Worthington,Zoe Saldana,Sig

## 2. Building a DataFrame With pandas 
Put the data into data frame with Pandas

In [6]:
num_columns = 13
num_rows = len(data) // num_columns
data_2d = [data[i*num_columns : (i+1)*num_columns] for i in range(num_rows)]
data_2d
df = pd.DataFrame(data_2d, columns=['Rank', 'Title', 'Year', 'Rating', 'Runtime', 'Genre', 'IMDB_Score', 'Metascore', 'Director', 'Stars', 'Description' ,'Votes','Gross(M)'])
df

,Rank,Title,Year,Rating,Runtime,Genre,IMDB_Score,Metascore,Director,Stars,Description,Votes,Gross(M)
0,1.,Spider-Man: Across the Spider-Verse,(2023),PG,140 min,"Animation, Action, Adventure",8.9,86,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...","Shameik Moore,Hailee Steinfeld,Brian Tyree ...","Miles Morales catapults across the Multiverse,...","169,962",NaN
1,2.,Titanic,(1997),PG-13,194 min,"Drama, Romance",7.9,75,James Cameron,"Leonardo DiCaprio,Kate Winslet,Billy Zane,K...",A seventeen-year-old aristocrat falls in love ...,"1,228,149",$659.33M
2,3.,Avatar: The Way of Water,(2022),PG-13,192 min,"Action, Adventure, Fantasy",7.6,67,James Cameron,"Sam Worthington,Zoe Saldana,Sigourney Weave...",Jake Sully lives with his newfound family form...,"425,575",$659.68M
3,4.,John Wick: Chapter 4,(2023),R,169 min,"Action, Crime, Thriller",7.9,78,Chad Stahelski,"Keanu Reeves,Laurence Fishburne,George Geor...",John Wick uncovers a path to defeating The Hig...,"231,771",NaN
4,5.,Indiana Jones and the Raiders of the Lost Ark,(1981),PG,115 min,"Action, Adventure",8.4,85,Steven Spielberg,"Harrison Ford,Karen Allen,Paul Freeman,John...","In 1936, archaeologist and adventurer Indiana ...","998,423",$248.16M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996.,Vicky Donor,(2012),Not Rated,126 min,"Comedy, Romance",7.8,NaN,Shoojit Sircar,"Ayushmann Khurrana,Yami Gautam,Annu Kapoor,...",A man is brought in by an infertility doctor t...,"44,434",$0.17M
996,997.,Vizontele,(2001),Not Rated,110 min,"Comedy, Drama",8.0,NaN,"Yilmaz Erdogan, Ömer Faruk Sorak","Yilmaz Erdogan,Demet Akbag,Altan Erkekli,Ce...","Lives of residents in a small, Anatolian villa...","37,767",NaN
997,998.,Sarfarosh,(1999),Not Rated,174 min,"Action, Drama, Thriller",8.1,NaN,John Mathew Matthan,"Ali Khan,Akhilendra Mishra,Makrand Deshpand...",After his brother is killed and father severel...,"26,295",NaN
998,999.,Airlift,(2016),Not Rated,130 min,"Action, Drama, History",7.9,NaN,Raja Menon,"Akshay Kumar,Nimrat Kaur,Kumud Mishra,Praka...","When Iraq invades Kuwait in August 1990, a cal...","57,934",NaN


## 3. Data Cleaning

Data cleaning - remove the '()' from year

In [7]:
df['Year']=df['Year'].str.replace('(', '').str.replace(')', '')
df

C:\Users\Asus\AppData\Local\Temp\ipykernel_22684\2843403066.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Year']=df['Year'].str.replace('(', '').str.replace(')', '')


,Rank,Title,Year,Rating,Runtime,Genre,IMDB_Score,Metascore,Director,Stars,Description,Votes,Gross(M)
0,1.,Spider-Man: Across the Spider-Verse,2023,PG,140 min,"Animation, Action, Adventure",8.9,86,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...","Shameik Moore,Hailee Steinfeld,Brian Tyree ...","Miles Morales catapults across the Multiverse,...","169,962",NaN
1,2.,Titanic,1997,PG-13,194 min,"Drama, Romance",7.9,75,James Cameron,"Leonardo DiCaprio,Kate Winslet,Billy Zane,K...",A seventeen-year-old aristocrat falls in love ...,"1,228,149",$659.33M
2,3.,Avatar: The Way of Water,2022,PG-13,192 min,"Action, Adventure, Fantasy",7.6,67,James Cameron,"Sam Worthington,Zoe Saldana,Sigourney Weave...",Jake Sully lives with his newfound family form...,"425,575",$659.68M
3,4.,John Wick: Chapter 4,2023,R,169 min,"Action, Crime, Thriller",7.9,78,Chad Stahelski,"Keanu Reeves,Laurence Fishburne,George Geor...",John Wick uncovers a path to defeating The Hig...,"231,771",NaN
4,5.,Indiana Jones and the Raiders of the Lost Ark,1981,PG,115 min,"Action, Adventure",8.4,85,Steven Spielberg,"Harrison Ford,Karen Allen,Paul Freeman,John...","In 1936, archaeologist and adventurer Indiana ...","998,423",$248.16M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996.,Vicky Donor,2012,Not Rated,126 min,"Comedy, Romance",7.8,NaN,Shoojit Sircar,"Ayushmann Khurrana,Yami Gautam,Annu Kapoor,...",A man is brought in by an infertility doctor t...,"44,434",$0.17M
996,997.,Vizontele,2001,Not Rated,110 min,"Comedy, Drama",8.0,NaN,"Yilmaz Erdogan, Ömer Faruk Sorak","Yilmaz Erdogan,Demet Akbag,Altan Erkekli,Ce...","Lives of residents in a small, Anatolian villa...","37,767",NaN
997,998.,Sarfarosh,1999,Not Rated,174 min,"Action, Drama, Thriller",8.1,NaN,John Mathew Matthan,"Ali Khan,Akhilendra Mishra,Makrand Deshpand...",After his brother is killed and father severel...,"26,295",NaN
998,999.,Airlift,2016,Not Rated,130 min,"Action, Drama, History",7.9,NaN,Raja Menon,"Akshay Kumar,Nimrat Kaur,Kumud Mishra,Praka...","When Iraq invades Kuwait in August 1990, a cal...","57,934",NaN


Data cleaning - remove the min from the timemin value

In [8]:
df['Runtime']=df['Runtime'].str.replace(' min', '')
df

,Rank,Title,Year,Rating,Runtime,Genre,IMDB_Score,Metascore,Director,Stars,Description,Votes,Gross(M)
0,1.,Spider-Man: Across the Spider-Verse,2023,PG,140,"Animation, Action, Adventure",8.9,86,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...","Shameik Moore,Hailee Steinfeld,Brian Tyree ...","Miles Morales catapults across the Multiverse,...","169,962",NaN
1,2.,Titanic,1997,PG-13,194,"Drama, Romance",7.9,75,James Cameron,"Leonardo DiCaprio,Kate Winslet,Billy Zane,K...",A seventeen-year-old aristocrat falls in love ...,"1,228,149",$659.33M
2,3.,Avatar: The Way of Water,2022,PG-13,192,"Action, Adventure, Fantasy",7.6,67,James Cameron,"Sam Worthington,Zoe Saldana,Sigourney Weave...",Jake Sully lives with his newfound family form...,"425,575",$659.68M
3,4.,John Wick: Chapter 4,2023,R,169,"Action, Crime, Thriller",7.9,78,Chad Stahelski,"Keanu Reeves,Laurence Fishburne,George Geor...",John Wick uncovers a path to defeating The Hig...,"231,771",NaN
4,5.,Indiana Jones and the Raiders of the Lost Ark,1981,PG,115,"Action, Adventure",8.4,85,Steven Spielberg,"Harrison Ford,Karen Allen,Paul Freeman,John...","In 1936, archaeologist and adventurer Indiana ...","998,423",$248.16M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996.,Vicky Donor,2012,Not Rated,126,"Comedy, Romance",7.8,NaN,Shoojit Sircar,"Ayushmann Khurrana,Yami Gautam,Annu Kapoor,...",A man is brought in by an infertility doctor t...,"44,434",$0.17M
996,997.,Vizontele,2001,Not Rated,110,"Comedy, Drama",8.0,NaN,"Yilmaz Erdogan, Ömer Faruk Sorak","Yilmaz Erdogan,Demet Akbag,Altan Erkekli,Ce...","Lives of residents in a small, Anatolian villa...","37,767",NaN
997,998.,Sarfarosh,1999,Not Rated,174,"Action, Drama, Thriller",8.1,NaN,John Mathew Matthan,"Ali Khan,Akhilendra Mishra,Makrand Deshpand...",After his brother is killed and father severel...,"26,295",NaN
998,999.,Airlift,2016,Not Rated,130,"Action, Drama, History",7.9,NaN,Raja Menon,"Akshay Kumar,Nimrat Kaur,Kumud Mishra,Praka...","When Iraq invades Kuwait in August 1990, a cal...","57,934",NaN


Data cleaning - remove the $ and M from the data value

In [9]:
df['Gross(M)']=df['Gross(M)'].str.replace('$', '').str.replace('M','')
df

C:\Users\Asus\AppData\Local\Temp\ipykernel_22684\2748443626.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Gross(M)']=df['Gross(M)'].str.replace('$', '').str.replace('M','')


,Rank,Title,Year,Rating,Runtime,Genre,IMDB_Score,Metascore,Director,Stars,Description,Votes,Gross(M)
0,1.,Spider-Man: Across the Spider-Verse,2023,PG,140,"Animation, Action, Adventure",8.9,86,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...","Shameik Moore,Hailee Steinfeld,Brian Tyree ...","Miles Morales catapults across the Multiverse,...","169,962",NaN
1,2.,Titanic,1997,PG-13,194,"Drama, Romance",7.9,75,James Cameron,"Leonardo DiCaprio,Kate Winslet,Billy Zane,K...",A seventeen-year-old aristocrat falls in love ...,"1,228,149",659.33
2,3.,Avatar: The Way of Water,2022,PG-13,192,"Action, Adventure, Fantasy",7.6,67,James Cameron,"Sam Worthington,Zoe Saldana,Sigourney Weave...",Jake Sully lives with his newfound family form...,"425,575",659.68
3,4.,John Wick: Chapter 4,2023,R,169,"Action, Crime, Thriller",7.9,78,Chad Stahelski,"Keanu Reeves,Laurence Fishburne,George Geor...",John Wick uncovers a path to defeating The Hig...,"231,771",NaN
4,5.,Indiana Jones and the Raiders of the Lost Ark,1981,PG,115,"Action, Adventure",8.4,85,Steven Spielberg,"Harrison Ford,Karen Allen,Paul Freeman,John...","In 1936, archaeologist and adventurer Indiana ...","998,423",248.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996.,Vicky Donor,2012,Not Rated,126,"Comedy, Romance",7.8,NaN,Shoojit Sircar,"Ayushmann Khurrana,Yami Gautam,Annu Kapoor,...",A man is brought in by an infertility doctor t...,"44,434",0.17
996,997.,Vizontele,2001,Not Rated,110,"Comedy, Drama",8.0,NaN,"Yilmaz Erdogan, Ömer Faruk Sorak","Yilmaz Erdogan,Demet Akbag,Altan Erkekli,Ce...","Lives of residents in a small, Anatolian villa...","37,767",NaN
997,998.,Sarfarosh,1999,Not Rated,174,"Action, Drama, Thriller",8.1,NaN,John Mathew Matthan,"Ali Khan,Akhilendra Mishra,Makrand Deshpand...",After his brother is killed and father severel...,"26,295",NaN
998,999.,Airlift,2016,Not Rated,130,"Action, Drama, History",7.9,NaN,Raja Menon,"Akshay Kumar,Nimrat Kaur,Kumud Mishra,Praka...","When Iraq invades Kuwait in August 1990, a cal...","57,934",NaN


Data cleaning - clear the ',' from the votes value

In [10]:
df['Votes']=df['Votes'].str.replace(',', '')
df

,Rank,Title,Year,Rating,Runtime,Genre,IMDB_Score,Metascore,Director,Stars,Description,Votes,Gross(M)
0,1.,Spider-Man: Across the Spider-Verse,2023,PG,140,"Animation, Action, Adventure",8.9,86,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...","Shameik Moore,Hailee Steinfeld,Brian Tyree ...","Miles Morales catapults across the Multiverse,...",169962,NaN
1,2.,Titanic,1997,PG-13,194,"Drama, Romance",7.9,75,James Cameron,"Leonardo DiCaprio,Kate Winslet,Billy Zane,K...",A seventeen-year-old aristocrat falls in love ...,1228149,659.33
2,3.,Avatar: The Way of Water,2022,PG-13,192,"Action, Adventure, Fantasy",7.6,67,James Cameron,"Sam Worthington,Zoe Saldana,Sigourney Weave...",Jake Sully lives with his newfound family form...,425575,659.68
3,4.,John Wick: Chapter 4,2023,R,169,"Action, Crime, Thriller",7.9,78,Chad Stahelski,"Keanu Reeves,Laurence Fishburne,George Geor...",John Wick uncovers a path to defeating The Hig...,231771,NaN
4,5.,Indiana Jones and the Raiders of the Lost Ark,1981,PG,115,"Action, Adventure",8.4,85,Steven Spielberg,"Harrison Ford,Karen Allen,Paul Freeman,John...","In 1936, archaeologist and adventurer Indiana ...",998423,248.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996.,Vicky Donor,2012,Not Rated,126,"Comedy, Romance",7.8,NaN,Shoojit Sircar,"Ayushmann Khurrana,Yami Gautam,Annu Kapoor,...",A man is brought in by an infertility doctor t...,44434,0.17
996,997.,Vizontele,2001,Not Rated,110,"Comedy, Drama",8.0,NaN,"Yilmaz Erdogan, Ömer Faruk Sorak","Yilmaz Erdogan,Demet Akbag,Altan Erkekli,Ce...","Lives of residents in a small, Anatolian villa...",37767,NaN
997,998.,Sarfarosh,1999,Not Rated,174,"Action, Drama, Thriller",8.1,NaN,John Mathew Matthan,"Ali Khan,Akhilendra Mishra,Makrand Deshpand...",After his brother is killed and father severel...,26295,NaN
998,999.,Airlift,2016,Not Rated,130,"Action, Drama, History",7.9,NaN,Raja Menon,"Akshay Kumar,Nimrat Kaur,Kumud Mishra,Praka...","When Iraq invades Kuwait in August 1990, a cal...",57934,NaN


## 4. Display Cleaned and Converted Code in Pandas

In [11]:
df

,Rank,Title,Year,Rating,Runtime,Genre,IMDB_Score,Metascore,Director,Stars,Description,Votes,Gross(M)
0,1.,Spider-Man: Across the Spider-Verse,2023,PG,140,"Animation, Action, Adventure",8.9,86,"Joaquim Dos Santos, Kemp Powers, Justin K. Tho...","Shameik Moore,Hailee Steinfeld,Brian Tyree ...","Miles Morales catapults across the Multiverse,...",169962,NaN
1,2.,Titanic,1997,PG-13,194,"Drama, Romance",7.9,75,James Cameron,"Leonardo DiCaprio,Kate Winslet,Billy Zane,K...",A seventeen-year-old aristocrat falls in love ...,1228149,659.33
2,3.,Avatar: The Way of Water,2022,PG-13,192,"Action, Adventure, Fantasy",7.6,67,James Cameron,"Sam Worthington,Zoe Saldana,Sigourney Weave...",Jake Sully lives with his newfound family form...,425575,659.68
3,4.,John Wick: Chapter 4,2023,R,169,"Action, Crime, Thriller",7.9,78,Chad Stahelski,"Keanu Reeves,Laurence Fishburne,George Geor...",John Wick uncovers a path to defeating The Hig...,231771,NaN
4,5.,Indiana Jones and the Raiders of the Lost Ark,1981,PG,115,"Action, Adventure",8.4,85,Steven Spielberg,"Harrison Ford,Karen Allen,Paul Freeman,John...","In 1936, archaeologist and adventurer Indiana ...",998423,248.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996.,Vicky Donor,2012,Not Rated,126,"Comedy, Romance",7.8,NaN,Shoojit Sircar,"Ayushmann Khurrana,Yami Gautam,Annu Kapoor,...",A man is brought in by an infertility doctor t...,44434,0.17
996,997.,Vizontele,2001,Not Rated,110,"Comedy, Drama",8.0,NaN,"Yilmaz Erdogan, Ömer Faruk Sorak","Yilmaz Erdogan,Demet Akbag,Altan Erkekli,Ce...","Lives of residents in a small, Anatolian villa...",37767,NaN
997,998.,Sarfarosh,1999,Not Rated,174,"Action, Drama, Thriller",8.1,NaN,John Mathew Matthan,"Ali Khan,Akhilendra Mishra,Makrand Deshpand...",After his brother is killed and father severel...,26295,NaN
998,999.,Airlift,2016,Not Rated,130,"Action, Drama, History",7.9,NaN,Raja Menon,"Akshay Kumar,Nimrat Kaur,Kumud Mishra,Praka...","When Iraq invades Kuwait in August 1990, a cal...",57934,NaN


## 5. Saving Your Data to a CSV

In [12]:
df.to_csv('C://Users/Asus/Desktop/ds1.1/Data files/IMDB_Data.csv', index=False)

## 6. Conclusion
What have you leanrt from this practice?